In [42]:
import nltk
import gensim
import numpy as np
import pandas as pd

import pickle

from nltk.corpus import gutenberg, brown, reuters, stopwords

from collections import Counter, defaultdict
from pprint import pprint
from functools import reduce
from operator import itemgetter

from utils.nltk_utils import *
from utils.ngrams import *
from utils.grammar import parse_phrases
from utils.eval import get_test_keywords
stop_words = stopwords.words('english')

In [3]:
import traceback
from typing import List, Tuple

from tqdm import tqdm

from utils.grammar_utils import tags_seq_to_symbols
from utils.io import Cache
from utils.ngrams import ngram2str


observed_tags = Cache.load_observed_tags()
terminal_rules = Cache.load_terminal_rules()


def parse_phrases(tt_ngrams) -> Tuple[List, List[List[Tuple]]]:
    global observed_tags, terminal_rules
    
    phrases = []
    phrases_types = []

    if observed_tags is None:
        observed_tags = dict()

    for tt_gram in tt_ngrams:
        symbols = tuple(tags_seq_to_symbols([tag
                                             for _, tag in tt_gram]))

        phrase = tt_gram

        # check if tags phrase has been already observed
        tags_str = ngram2str(symbols)

        if tags_str in observed_tags:
            if observed_tags[tags_str] is not None:
                phrases.append(phrase)
                phrases_types.append(observed_tags[tags_str])

            continue

        p_types_dict = terminal_rules.get(tags_str)

        if p_types_dict:
            p_type = max(p_types_dict, key=lambda k: p_types_dict[k])

            phrases.append(phrase)
            phrases_types.append(p_type)

            observed_tags[tags_str] = p_type

        else:
            observed_tags[tags_str] = None

#     try:
#         Cache.save_observed_tags(observed_tags)

#     except Exception as e:
#         traceback.print_exc(e)

    return phrases_types, phrases


# Preparing corpora

## Load data from NLTK 

In [4]:
sents = gutenberg.sents() + brown.sents() + reuters.sents()

In [5]:
sents = [tuple(s) for s in sents]

words = [w.lower() 
         for s in sents for w in s]

vocab = sorted(list(set(words)))

print('Sents:', len(sents))
print('Words:', len(words))
print('Vocab:', len(vocab))

Sents: 210608
Words: 5503894
Vocab: 87046


In [6]:
# creating indexes

# {sentence: sent_index}
sents_ind = {sents[i]: i for i in range(len(sents))}

# {word: word_index}
vocab_ind = {vocab[j]: j for j in range(len(vocab))}

## Create mapping from words to sentences

In [7]:
stemmer = Stemmer()

stems = [stemmer.stem(word) for word in vocab]

# {word_index: stem}
stems_map = {vocab_ind[word]: stem 
             for word, stem in zip(vocab, stems)}

print('Stems:', len(set(stems)))

Stems: 58858


In [8]:
stem2word = {stem: vocab[ind]
             for ind, stem in stems_map.items()}

## Parsing corpora

In [ ]:
# load

with open('data/cache/stems_phrases', mode='rb') as fp:
    stems_phrases = pickle.load(fp)

In [9]:
parser = NGramsParser()

sents_words_indexes = parser.parse_sents_tokens(sents)

In [10]:
# {stem: Set[sentence_ind]}
stems_phrases = defaultdict(dict)

for i, (s_words, words_indexes) in tqdm(enumerate(zip(sents, sents_words_indexes), start=1)):
    words_ttokens = nltk.pos_tag([w.lower() for w in s_words])
    
    tt_ngrams = [ngr
                 for i in range(1, 5 + 1) 
                 for ngr in n_grams(words_ttokens, i, words_indexes, pad_left=False)]

    types, phrases = parse_phrases(tt_ngrams)

    # format and store phrases
    for t, p in zip(types, phrases):
        phrase_inds = tuple(vocab_ind[token] for token, _ in p)
        
        for word_ind in phrase_inds:
            if vocab[word_ind] not in stop_words:
                stem = stems_map[word_ind]
                
                if not stems_phrases[stem].get(t):
                    stems_phrases[stem][t] = defaultdict(set)
                
                stem_phr_t = stems_phrases[stem][t]
                
                stem_phr_t[phrase_inds].add(sents_ind[s_words])

210608it [07:18, 480.01it/s]


In [11]:
def get_phrases(word, stemmed=True):
    if stemmed:
        stem = word
    else:
        stem = stemmer.stem(word)
        
    phrs = stems_phrases[stem]
    
    if not phrs:
        return {}
    
    phrases_ = defaultdict(set)
    
    for p_type, phr_dict in phrs.items():
        for phrase, sents in phr_dict.items():
            phrase_ = tuple(vocab[ind] for ind in phrase)
            
            phrases_[p_type].add(phrase_)
            
    return phrases_

In [12]:
# save

with open('data/cache/stems_phrases', mode='wb') as fp:
    pickle.dump(stems_phrases, fp)

## Word2Vec model

In [25]:
# loading
model = gensim.models.Word2Vec.load('data/w2v/CBOW_300_10_all')

In [14]:
sents_stemmed = [[stems_map[vocab_ind[w.lower()]] for w in s] for s in sents]

In [33]:
model = gensim.models.Word2Vec(sentences=sents_stemmed, size=300, window=15, min_count=1, hs=1, negative=0)
model.save('data/w2v/CBOW_300_10_hs_all')

In [34]:
# testing

print(model.wv.most_similar(positive=[stemmer.stem('woman'), stemmer.stem('king')], negative=[stemmer.stem('man')]))
print(model.wv.similarity(stemmer.stem('campus'), stemmer.stem('dormitory')))
print(model.wv.doesnt_match([stemmer.stem(w) for w in "dormitory bring campus".split()]))

[('queen', 0.3553354740142822), ('daughter', 0.35418325662612915), ('esther', 0.3361766040325165), ('husband', 0.3246108889579773), ('absalom', 0.3200189769268036), ('mordecai', 0.31594496965408325), ('samaria', 0.3088769316673279), ('vashti', 0.30666205286979675), ('hebron', 0.298782080411911), ('selleth', 0.29791125655174255)]
0.20989265750352995
bring


In [35]:
w2v = model.wv

In [19]:
del model

## Count number of phrases

In [121]:
# data for analysis

phrases_count = defaultdict(dict)

phrases_lengths_count = defaultdict(int)
phrases_types_count = defaultdict(int)

stems_phrases_count = defaultdict(dict)

for stem, stem_phrases_dict in stems_phrases.items():
    for phrase_type, phrases_dict in stem_phrases_dict.items():
#         stem_phrase_type_count = sum([len(s_ids) for s_ids in phrases_dict.values()])
        for phrase_tuple, s_ids in phrases_dict.items():
            phrase_len = len(phrase_tuple)
            phrase_sents_count = len(s_ids)
            
            if not stems_phrases_count[stem].get(phrase_len):
                stems_phrases_count[stem][phrase_len] = defaultdict(int)
                
            # count number of such phrase type for given stem
            stems_phrases_count[stem][phrase_len][phrase_type] += phrase_sents_count 

            # count total number of such phrase type
            phrases_count[phrase_len][phrase_type] = phrases_count[phrase_len].get(phrase_type, 0) + phrase_sents_count

            # count total number of all phrases
            phrases_lengths_count[phrase_len] += phrase_sents_count
            phrases_types_count[phrase_type] += phrase_sents_count

all_phrases_count = sum(phrases_types_count.values())

In [127]:
phrases_count

defaultdict(dict,
            {1: {'ADJP': 332093,
              'ADVP': 131822,
              'NP': 1396926,
              'PP': 41047,
              'VP': 550396},
             2: {'ADJP': 288142,
              'ADVP': 418156,
              'NP': 1985844,
              'PP': 3328,
              'VP': 66991},
             3: {'ADJP': 44770,
              'ADVP': 31064,
              'NP': 1353053,
              'PP': 7554,
              'VP': 8215},
             4: {'ADJP': 4179, 'ADVP': 223, 'NP': 429226},
             5: {'ADJP': 245, 'NP': 53025}})

In [118]:
stems_phrases_count['assum']

{1: defaultdict(int, {'ADJP': 4, 'ADVP': 2, 'NP': 4, 'PP': 3, 'VP': 304}),
 2: defaultdict(int, {'ADJP': 42, 'ADVP': 4, 'NP': 116, 'VP': 6}),
 3: defaultdict(int, {'NP': 27, 'VP': 1}),
 4: defaultdict(int, {'NP': 7}),
 5: defaultdict(int, {'NP': 1})}

In [128]:
def get_word_phrases(word, stemmed=True):
    """
        Return sorted log probs of phrase types and length for the word 
    """
    if not stemmed:
        stem = stemmer.stem(word)
    else:
        stem = word
    
    if not stems_phrases_count.get(stem):
        return []
    
    stem_phrases = stems_phrases_count[stem]
    
    phrases_probs = []
    
    for p_length, p_types_count in stem_phrases.items():
        for p_type, p_type_count in p_types_count.items():
            prob_len = phrases_lengths_count[p_length] / all_phrases_count
            
            prob_len_type = phrases_count[p_length][p_type] / phrases_lengths_count[p_length]
            
            prob_word_len_type = p_type_count / phrases_count[p_length][p_type]
            
            log_prob = sum(np.log(p) for p in [prob_word_len_type, prob_len_type, prob_len])
            
            phrases_probs.append(((p_type, p_length), log_prob))
    
    return sorted(phrases_probs, key=itemgetter(1), reverse=True)

get_word_phrases('assume', stemmed=False)

[(('VP', 1), -10.065077456873636),
 (('NP', 2), -11.028514967173495),
 (('ADJP', 2), -12.044435539996488),
 (('NP', 3), -12.486268292275527),
 (('NP', 4), -13.836195009224543),
 (('VP', 2), -13.990345689051804),
 (('NP', 1), -14.395810797159967),
 (('ADJP', 1), -14.395810797159967),
 (('ADVP', 2), -14.395810797159967),
 (('PP', 1), -14.683492869611747),
 (('ADVP', 1), -15.088957977719911),
 (('VP', 3), -15.782105158279858),
 (('NP', 5), -15.782105158279858)]

## Scoring phrases

In [129]:
phrases_ = get_phrases('assume', stemmed=False)

all_scored_phrases = defaultdict(dict)

for p_type, phrases in phrases_.items():
    phrases_ = list(phrases)
    
    scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                   for phr in phrases_]))
#     scores = np.exp(scores - np.max(scores, axis=0))
#     scores = np.log(scores / scores.sum(axis=0))
    for phr, sc in zip(phrases_, scores):
        if not all_scored_phrases[p_type].get(len(phr)):
            all_scored_phrases[p_type][len(phr)] = []
    
        all_scored_phrases[p_type][len(phr)].append((phr, sc))

In [130]:
for p_type, scored_phrases in all_scored_phrases.items():
    print('%s\n' % p_type)
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=True)[:5]

        best_phr = [(' '.join(p), s) for p, s in best_phr]

        print(n)
        pprint(best_phr)
        print()

NP

1
[('assumes', -9.87307), ('assume', -9.87307)]

2
[('that assumed', -9.019429),
 ('assume that', -9.019429),
 ('assume man', -9.951341),
 ('assume office', -10.356435),
 ('assume the', -11.597148)]

3
[('assuming crude oil', -16.74822),
 ('assuming oil prices', -17.29162),
 ('an assumed air', -17.8685),
 ('assuming market conditions', -17.922554),
 ('the question assumed', -19.673552)]

4
[('an assumed air of', -21.883614),
 ('with an assumed air', -22.628468),
 ('and an assumed impact', -25.068672),
 ('under an assumed name', -25.50867),
 ('an assumed impact velocity', -29.45615)]

5
[('assumes some other horrible forme', -39.57282)]

ADVP

1
[('assumes', -9.87307), ('assume', -9.87307)]

2
[('assume the', -11.597148),
 ('this assumes', -15.635136),
 ('shrs assume', -21.739788),
 ('there assumes', -22.380066)]

VP

1
[('assumes', -9.87307),
 ('assumed', -9.87307),
 ('assume', -9.87307),
 ('assuming', -9.87307)]

2
[('assuming that', -9.019429),
 ('assuming for', -17.421629),
 ('a

In [51]:
for i in range(1, 5 + 1):
    nps = scored_nps[i]
    best_nps = sorted(nps, key=itemgetter(1), reverse=True)[:5]
    
    best_nps = [(' '.join(p), s) for p, s in best_nps]
    
    print(i)
    pprint(best_nps)
    print()

1
[('advice', -10.578289)]

2
[('my advice', -10.6507635),
 ('give advice', -11.619597),
 ('wanted advice', -13.039914),
 ('and advice', -13.891079),
 ('him advice', -13.957809)]

3
[('such advice as', -12.637161),
 ('of the advice', -18.26074),
 ('the advice of', -18.26074),
 ('their advice for', -18.533184),
 ('their advice ."', -18.64547)]

4
[('for some other advice', -23.814346),
 ('your advice and counsel', -25.596104),
 ('her kindness and advice', -27.856691),
 ('the free advice of', -28.051847),
 ("the hetman's advice of", -29.096107)]

5
[('the close-in support and advice', -41.098873)]



## Working with keywords

In [20]:
kws = get_test_keywords('data/lingualeo_words.csv')

print('Keywords: ', len(kws))

Keywords:  874


In [21]:
TEST_SIZE = 5
np.random.seed = 0

all_kws_found = False

while not all_kws_found:    
    test_kws = list(np.random.choice(list(kws), size=TEST_SIZE, replace=False))

    print('Test keywords:', test_kws)

    test_kws = [stemmer.stem(kw) for kw in test_kws]
    kws_phrases = [stems_phrases.get(kw) for kw in test_kws]
    print('Test keywords stems:', test_kws)

    all_kws_found = all(kws_phrases)
    
    print('All kws found:', all_kws_found)
    print()

Test keywords: ['advice', 'drab', 'shame', 'proclivity', 'admissible']
Test keywords stems: ['advic', 'drab', 'shame', 'procliv', 'admiss']
All kws found: True



In [22]:
def get_pairs_counter(test_kws):
    pairs = []

    for i, kw in enumerate(test_kws):
        cp = list(test_kws)
        cp.remove(kw)

        dsts = w2v.distances(kw, cp)
        print(kw, dsts)
        max_ind = np.argmin(dsts)
        print(i, max_ind)
        sim_kw = test_kws[max_ind + (1 if i <= max_ind else 0)]
        print(kw, sim_kw)

        pair = (kw, sim_kw)

        if (sim_kw, kw) in pairs:
            pair = (sim_kw, kw)

        pairs.append(pair)

    return Counter(pairs)

get_pairs_counter(test_kws)

advic [1.0484983  0.5876609  0.67798024 0.6448846 ]
0 1
advic shame
drab [1.0484983  0.85772705 0.5455078  0.85652786]
1 2
drab procliv
shame [0.5876609  0.85772705 0.9682493  0.80564225]
2 0
shame advic
procliv [0.67798024 0.5455078  0.9682493  0.66142946]
3 1
procliv drab
admiss [0.6448846  0.85652786 0.8056423  0.66142946]
4 0
admiss advic


Counter({('admiss', 'advic'): 1,
         ('advic', 'shame'): 2,
         ('drab', 'procliv'): 2})

## Using processed corpora

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)
# stems_phrases[s]
vocab_ind['grievance']

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)

ss = []
for k, v in stems_phrases[s]['NP'].items():
    ss.extend([s_id for s_id in v])
    
Counter(ss)